In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/child-mind-institute-detect-sleep-states/train_series.parquet
/kaggle/input/child-mind-institute-detect-sleep-states/sample_submission.csv
/kaggle/input/child-mind-institute-detect-sleep-states/train_events.csv
/kaggle/input/child-mind-institute-detect-sleep-states/test_series.parquet
/kaggle/input/models/bestXGboost.pkl
/kaggle/input/models/enmoScaler.pkl
/kaggle/input/models/anglezScaler.pkl
/kaggle/input/models/labelEncoder.pkl
/kaggle/input/preprocesseddata/cropedTrainSeries3_2.csv
/kaggle/input/bestmodels/bestXGboost.joblib
/kaggle/input/bestmodels/labelEncoder_2.joblib


In [2]:
import joblib
import numpy as np
from datetime import datetime as dt
from datetime import timedelta
from datetime import time as tm

import gc

from xgboost import XGBClassifier


from sklearn.preprocessing import MinMaxScaler

In [3]:
XGboostModel2=loaded_model = joblib.load('/kaggle/input/bestmodels/bestXGboost.joblib')
labelEncoder=joblib.load('/kaggle/input/bestmodels/labelEncoder_2.joblib')
enmoScaler=joblib.load('/kaggle/input/models/enmoScaler.pkl')
anglezScaler=joblib.load('/kaggle/input/models/anglezScaler.pkl')
testDF=pd.read_parquet('/kaggle/input/child-mind-institute-detect-sleep-states/test_series.parquet')

In [4]:
test_series_DF_CP=testDF.copy()

In [5]:
test_series_DF_CP.head()

,series_id,step,timestamp,anglez,enmo
0,038441c925bb,0,2018-08-14T15:30:00-0400,2.6367,0.0217
1,038441c925bb,1,2018-08-14T15:30:05-0400,2.6368,0.0215
2,038441c925bb,2,2018-08-14T15:30:10-0400,2.6370,0.0216
3,038441c925bb,3,2018-08-14T15:30:15-0400,2.6368,0.0213
4,038441c925bb,4,2018-08-14T15:30:20-0400,2.6368,0.0215


In [6]:
test_series_DF_CP['timestamp'] = pd.to_datetime(test_series_DF_CP['timestamp'],utc=True)
test_series_DF_CP['Date'] = test_series_DF_CP['timestamp'].dt.date
test_series_DF_CP['Time'] = test_series_DF_CP['timestamp'].dt.time

In [7]:
test_series_DF_CP.head()

,series_id,step,timestamp,anglez,enmo,Date,Time
0,038441c925bb,0,2018-08-14 19:30:00+00:00,2.6367,0.0217,2018-08-14,19:30:00
1,038441c925bb,1,2018-08-14 19:30:05+00:00,2.6368,0.0215,2018-08-14,19:30:05
2,038441c925bb,2,2018-08-14 19:30:10+00:00,2.6370,0.0216,2018-08-14,19:30:10
3,038441c925bb,3,2018-08-14 19:30:15+00:00,2.6368,0.0213,2018-08-14,19:30:15
4,038441c925bb,4,2018-08-14 19:30:20+00:00,2.6368,0.0215,2018-08-14,19:30:20


In [8]:
test_series_DF_CP['NewDate'] = pd.to_datetime(test_series_DF_CP['Date'])
# Create a new 'Season' column based on the month of each date
test_series_DF_CP['Season'] = test_series_DF_CP['NewDate'].dt.month.map(lambda x: {3: 'Spring', 4: 'Spring', 5: 'Spring',
                                                 6: 'Summer', 7: 'Summer', 8: 'Summer',
                                                 9: 'Autumn', 10: 'Autumn', 11: 'Autumn',
                                                 12: 'Winter', 1: 'Winter', 2: 'Winter'}.get(x))

# Use get_dummies to one-hot encode the 'Season' column
season_dummiesTest = test_series_DF_CP['Season'].apply(lambda x: pd.Series({x: 1}))
test_series_DF_CP = pd.concat([test_series_DF_CP, season_dummiesTest], axis=1)


In [9]:
test_series_DF_CP.head()

,series_id,step,timestamp,anglez,enmo,Date,Time,NewDate,Season,Summer,Spring,Winter
0,038441c925bb,0,2018-08-14 19:30:00+00:00,2.6367,0.0217,2018-08-14,19:30:00,2018-08-14,Summer,1.0,NaN,NaN
1,038441c925bb,1,2018-08-14 19:30:05+00:00,2.6368,0.0215,2018-08-14,19:30:05,2018-08-14,Summer,1.0,NaN,NaN
2,038441c925bb,2,2018-08-14 19:30:10+00:00,2.6370,0.0216,2018-08-14,19:30:10,2018-08-14,Summer,1.0,NaN,NaN
3,038441c925bb,3,2018-08-14 19:30:15+00:00,2.6368,0.0213,2018-08-14,19:30:15,2018-08-14,Summer,1.0,NaN,NaN
4,038441c925bb,4,2018-08-14 19:30:20+00:00,2.6368,0.0215,2018-08-14,19:30:20,2018-08-14,Summer,1.0,NaN,NaN


In [10]:
seasons=['Summer','Autumn','Spring','Winter']
for season in seasons:
    if season not in test_series_DF_CP.columns:
        test_series_DF_CP[season]=float('nan')
        
        
test_series_DF_CP['Date'] = pd.to_datetime(test_series_DF_CP['Date'])

# Add a new column 'weekday' to the DataFrame
test_series_DF_CP['weekday'] = test_series_DF_CP['Date'].dt.dayofweek

# Map the weekdays to 1 and weekends to 0
test_series_DF_CP['weekday'] = test_series_DF_CP['weekday'].apply(lambda x: 1 if x < 5 else 0)

test_series_DF_CP['anglezScaled']=anglezScaler.transform(test_series_DF_CP['anglez'].values.reshape(-1, 1))
test_series_DF_CP['enmoScaled']=anglezScaler.transform(test_series_DF_CP['enmo'].values.reshape(-1, 1))

In [11]:
test_series_DF_CP.isnull().sum()

series_id         0
step              0
timestamp         0
anglez            0
enmo              0
Date              0
Time              0
NewDate           0
Season            0
Summer          300
Spring          300
Winter          300
Autumn          450
weekday           0
anglezScaled      0
enmoScaled        0
dtype: int64

In [12]:
test_series_DF_CP['Date'] = pd.to_datetime(test_series_DF_CP['Date'])

# Add a new column 'weekday' to the DataFrame
test_series_DF_CP['weekday'] = test_series_DF_CP['Date'].dt.dayofweek

# Map the weekdays to 1 and weekends to 0
test_series_DF_CP['weekday'] = test_series_DF_CP['weekday'].apply(lambda x: 1 if x < 5 else 0)



In [13]:
test_series_DF_CP['anglezScaled']=anglezScaler.transform(test_series_DF_CP['anglez'].values.reshape(-1, 1))
test_series_DF_CP['enmoScaled']=anglezScaler.transform(test_series_DF_CP['enmo'].values.reshape(-1, 1))

In [14]:
test_series_DF_CP['Time'] = pd.to_datetime(test_series_DF_CP['Time'], format='%H:%M:%S')

#Extracting total seconds since midnight as a single feature
test_series_DF_CP['time_in_seconds'] = test_series_DF_CP['Time'].dt.hour * 3600 + test_series_DF_CP['Time'].dt.minute * 60 + test_series_DF_CP['Time'].dt.second

# Create empty columns for sin and cos
test_series_DF_CP['sin_time'] = np.nan
test_series_DF_CP['cos_time'] = np.nan

# Calculating sin and cos of time_in_seconds
test_series_DF_CP['sin_time'] = np.sin(2 * np.pi * test_series_DF_CP['time_in_seconds'] / 86400)  # 86400 seconds in a day
test_series_DF_CP['cos_time'] = np.cos(2 * np.pi * test_series_DF_CP['time_in_seconds'] / 86400)

In [15]:
trainFeatures=['sin_time','cos_time','Summer','Autumn','Spring','Winter','anglezScaled','enmoScaled','weekday']

In [16]:
#X_train=trainDF[trainFeatures]
#y_train=labelEncoder.transform(trainDF['event'])
X_test=test_series_DF_CP[trainFeatures]
#del trainDF
del test_series_DF_CP
gc.collect()

90

In [17]:
preditions=XGboostModel2.predict(X_test)
preditions

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [18]:
preditions_lables=labelEncoder.inverse_transform(preditions)


In [19]:
preditions_lables

array(['wakeup', 'wakeup', 'wakeup', 'wakeup', 'wakeup', 'wakeup',
       'wakeup', 'wakeup', 'wakeup', 'wakeup', 'wakeup', 'wakeup',
       'wakeup', 'wakeup', 'wakeup', 'wakeup', 'wakeup', 'wakeup',
       'wakeup', 'wakeup', 'wakeup', 'wakeup', 'wakeup', 'wakeup',
       'wakeup', 'wakeup', 'wakeup', 'wakeup', 'onset', 'onset', 'onset',
       'onset', 'onset', 'onset', 'onset', 'onset', 'onset', 'onset',
       'onset', 'onset', 'onset', 'onset', 'onset', 'onset', 'onset',
       'onset', 'onset', 'onset', 'onset', 'onset', 'onset', 'onset',
       'onset', 'onset', 'onset', 'onset', 'onset', 'onset', 'onset',
       'onset', 'onset', 'onset', 'onset', 'onset', 'onset', 'onset',
       'onset', 'onset', 'onset', 'onset', 'onset', 'onset', 'onset',
       'onset', 'onset', 'onset', 'onset', 'onset', 'onset', 'onset',
       'onset', 'onset', 'onset', 'onset', 'onset', 'onset', 'onset',
       'onset', 'onset', 'onset', 'onset', 'onset', 'onset', 'onset',
       'onset', 'onset', 'on

In [20]:
preditions_proba=XGboostModel2.predict_proba(X_test)
gc.collect()

0

In [21]:
submission=testDF[['series_id','step']]
submission['event']=preditions_lables
submission['score']=preditions_proba[:,1]

/tmp/ipykernel_20/2627425699.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  submission['event']=preditions_lables
/tmp/ipykernel_20/2627425699.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  submission['score']=preditions_proba[:,1]


In [22]:
#submission = submission.sort_values(['series_id','step']).reset_index(drop=True)
submission['row_id'] = submission.index.astype(int)
submission = submission[['row_id','series_id','step','event','score']]
#submission['score'] = submission['score'].fillna(submission['score'].mean())

In [23]:
submission.to_csv('submission.csv')